In [7]:
from itertools import product
import random
import pandas as pd
import numpy as np
import datetime
import time

class Quotation_Programmer:
    # Quotation Program을 구축하는 클래스
    # difficulty: int (1,2,3), shape: int (1=regular, 2=long or big, 세변의 합이 160cm 를 넘는가), 
    # weight: int(1: <5kg,2: <20kg,3 >=20kg, workload: int
    # 작업을 어려움, 아이템모양, 아이템무게를 조합해서 18개의 작업 종류를 정의
    difficulty_outcome = [1,2,3] # Work_definition의 각 튜플 요소의 인덱스 0
    shape_outcome = [1,2] # Work_definition의 각 튜플 요소의 인덱스 1
    weight_outcome = [1,2,3] # Work_definition의 각 튜플 요소의 인덱스 2
    work_definition = list(product(*[difficulty_outcome,shape_outcome,weight_outcome]))

    def team_formation(self):
#     supervisor_outcome = [0,1]
#     forkliftdriver_outcome = [0,1]
#     picker_outcome = [1,2,3,4]
#     packer_outcome = [1,2]
        self.supervisor_num = [0]*len(self.work_definition) #18가지의 업무단위 조합에 필요한 팀의 supervisor 수. 기본값: 0
        self.forkliftdriver_num = [0]*len(self.work_definition) #기본값: 0
        self.picker_num = [1]*len(self.work_definition) #기본값: 1
        self.packer_num = [1]*len(self.work_definition) #기본값: 1
    
        # 각 작업단위 특성을 고려하여 팀의 각 포지션별 필요 수 정하기
        for i in range(len(self.work_definition)):
            # 팀 내의 supervisor 수: 전체적인 업무강도 정도의 합이 4가 넘을 경우 +1
            if sum(self.work_definition [i]) > 4:
                self.supervisor_num[i]= 1
             # 팀 내의 forkliftdriver 수: 아이템 모양이 크거나 길때 즉 2일때 
            # 혹은 아이템이 30kg 이상으로 무거울 때 즉 3일때 +1    
            if (self.work_definition [i][1] > 1 or self.work_definition [i][2] > 2):
                self.forkliftdriver_num[i] = 1
            # 팀 내의 picker 수:
            ## weight = 3, shape = 2, difficulty =3 인경우 각각  picker수 +1
            if self.work_definition[i][2] == 3:
                self.picker_num[i] += 1
            if self.work_definition[i][1] == 2:
                self.picker_num[i] += 1
            if self.work_definition[i][0] == 3:
                self.picker_num[i] += 1
            # 팀 내의 packer 수:
            ## shape = 2, weight= 3 인경우 각각 packer수 +1 
            if self.work_definition[i][1] == 2:
                self.packer_num[i] += 1
            if self.work_definition[i][2] == 3:
                self.packer_num[i] += 1
        return self.supervisor_num, self.forkliftdriver_num, self.picker_num, self.packer_num
    
    def data_generation(self):
        #team_formation의 함수를 사용하여 team 구성하기
        self.team_formation_result = self.team_formation()
        self.supervisor_num = self.team_formation_result[0]
        self.forkliftdriver_num = self.team_formation_result[1]
        self.picker_num = self.team_formation_result[2]
        self.packer_num = self.team_formation_result[3]

        #Data Output 칼럼명 및 칼렴
        self.column_name = ['work_index','workload','s_mh/worklaod', 'f_mh/workload', 'pi_mh/workload', 'pa_mh/workload']
        self.work_index = [0] * len(self.work_definition) * 1000
        self.workload = [0] * len(self.work_definition) * 1000
        self.s_mh = [0] * len(self.work_definition) * 1000
        self.f_mh = [0] * len(self.work_definition) * 1000
        self.pi_mh = [0] * len(self.work_definition) * 1000
        self.pa_mh = [0] * len(self.work_definition) * 1000
        # random range로 쓸 작업단위 별 포지션의 한 사람당 예상 작업 range
        self.s_mh_range = [0]*len(self.work_definition) #기본값: 0
        self.f_mh_range = [0] *len(self.work_definition) #기본값: 0
        self.pi_mh_range = [[0.3,0.5]]*len(self.work_definition) #기본값: 0.3~0.5 manhour
        self.pa_mh_range = [[0.3,0.5]]*len(self.work_definition) #기본값: 0.3~0.5 manhour
        
        # 18가지 작업단위 에 대한 각 업무 포지션 별 한사람당 걸릴 것 같은 작업 시간 range 설정
        for i in range(len(self.work_definition)):
            
            # supervisor manhour range:
            # 필요인원이 0인 경우 0
            #if difficulty = 3 or (shape = 2 and weight = 3) supervisorMH range 0.4,0.7
            #else = 0.2~0.4
            if self.supervisor_num[i] == 0:
                pass

            elif self.work_definition[i][0] == 3 or (self.work_definition[i][1] == 2 and self.work_definition[i][2]):
                self.s_mh_range[i] = [0.4,0.7]
            else:
                self.s_mh_range[i] = [0.2,0.4]

            # forklift manhour range:
            # 필요인원이 0인 경우 0
            #if difficulty = 3 or shape = 2. forklift_MH_range 0.4,0.8 
            #else = 0.2~0.4
            if self.forkliftdriver_num[i] == 0:
                pass

            elif self.work_definition[i][0] == 3 or self.work_definition[i][1] == 2:
                self.f_mh_range[i] = [0.4,0.8]
            else:
                self.f_mh_range[i] = [0.2,0.4]

            # picker manhour range:
            # if difficulty >1 or shape = 2 or weight >1, 0.5~0.8
            # else 기본값
            if self.work_definition[i][0] >1 or self.work_definition[i][1] == 2 or self.work_definition[i][2] >1:
                self.pi_mh_range[i] = [0.5,0.8]

            # packer manhour range:
            # if difficulty >1 or shape = 2 or weight >1, 0.5~0.8
            # else 기본값
            if self.work_definition[i][0] >1 or self.work_definition[i][1] == 2 or self.work_definition[i][2] >1:
                self.pa_mh_range[i] = [0.5,0.8]

        # 18개의 작업단위에 대하여 각각 10000개의 workload 1~100의 랜덤정수,
        # 's_mh', 'f_mh', 'pi_mh', 'pa_mh'는 바로 위에 정의한 range에 맞춰서 랜덤float 곱하기
        # workload<5 1.05를 곱하기 *s_num*random *가중치 sum(work_definition[work_index])/10
        for case in range(1000):
            for i in range(len(self.work_definition)):
                self.work_index[i+(case*18)] = i
                self.workload[i+(case*18)] = random.randint(1,50)
                self.s_mh[i+(case*18)] = (0 if self.s_mh_range[i] == 0 
                                           else random.uniform(self.s_mh_range[i][0],self.s_mh_range[i][1]) *
                                          (1.05 if self.workload[i+(case*18)] > 5 else 1) *
                                          (sum(self.work_definition[i])/10) *
                                          self.supervisor_num[i])
                self.f_mh[i+(case*18)] = (0 if self.f_mh_range[i] == 0 
                                          else random.uniform(self.f_mh_range[i][0],self.f_mh_range[i][1]) *
                                          (1.05 if self.workload[i+(case*18)] > 5 else 1) *
                                          (sum(self.work_definition[i])/10) *
                                          self.forkliftdriver_num[i])
                self.pi_mh[i+(case*18)] = (random.uniform(self.pi_mh_range[i][0],self.pi_mh_range[i][1]) *
                                          (1.05 if self.workload[i+(case*18)] > 5 else 1) *
                                          (sum(self.work_definition[i])/10) *
                                          self.picker_num[i])
                self.pa_mh[i+(case*18)] = (random.uniform(self.pa_mh_range[i][0],self.pa_mh_range[i][1]) *
                                          (1.05 if self.workload[i+(case*18)] > 5 else 1) *
                                          (sum(self.work_definition[i])/10) *
                                          self.packer_num[i])

        # pandas로 dataset으로 변환한 후 csv로 출력. dataset을 리턴
        self.df = pd.DataFrame(zip(self.work_index, self.workload, self.s_mh, self.f_mh, self.pi_mh, self.pa_mh), \
                               columns = self.column_name)
        #print(self.df.head())
        self.df.to_csv('test1.csv')
        return self.df
    
    #인풋 데이터프레임을 토대로 각 케이스 별 평균 맨파워 도출
    def mean_manhour(self):
        self.df = self.data_generation()
        self.s_mh_mean = [0]*18
        self.f_mh_mean = [0]*18
        self.pi_mh_mean = [0]*18
        self.pa_mh_mean = [0]*18
        for i in range (18):
            self.s_mh_mean[i] = self.df.loc[self.df['work_index'] == i,'s_mh/worklaod'].mean()
            self.f_mh_mean[i] = self.df.loc[self.df['work_index'] == i,'f_mh/workload'].mean()
            self.pi_mh_mean[i] = self.df.loc[self.df['work_index'] == i,'pi_mh/workload'].mean()
            self.pa_mh_mean[i] = self.df.loc[self.df['work_index'] == i,'pa_mh/workload'].mean()
        return self.s_mh_mean, self.f_mh_mean, self.pi_mh_mean, self.pa_mh_mean


class Quotation(Quotation_Programmer):
    # Quotation_Programmer 상속, User가 Quotation 받고 싶은 아이템의 정보 input 및
    # 견적서 프린트 받아볼 수 있는 함수 포함
    def __init__(self):
        self.worktitle = input("견적 받고 싶은 아이템에 대하여 8가지 질문을 드릴려고 합니다.\
                                \n최대한 정확한 정보를 입력해주시면 감사하겠습니다.\n\
                                \nPick/Packing 작업 견적을 원하시는 아이템 이름을 입력하세요 >> ")
        while 1:
            self.fragile = input("입력하신 아이템이 깨지기 쉬운 유리/그릇 류 입니까? (y/n) >> ")
            try:
                if (self.fragile.lower() == 'y' or self.fragile.lower() == 'n'):
                    break
                else:
                    print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
            except:
                print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
        while 1:
            self.dangerous = input("입력하신 아이템이 위험화물인 배터리, 향수, 알코올, 화학약품 류 입니까? (y/n) >> ")
            try:
                if (self.dangerous.lower() == 'y' or self.dangerous.lower() == 'n'):
                    break
                else:
                    print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
            except:
                print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
        while 1:
            self.easy = input("입력하신 아이템이 가벼운 플라스틱, 금속, 종이 류 입니까? (y/n) >> ")
            try:
                if (self.easy.lower() == 'y' or self.easy.lower() == 'n'):
                    break
                else:
                    print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
            except:
                print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
        while 1:
            self.size = input("입력하신 아이템의 가로+세로+높이 합이 160cm을 넘거나 또는 한변의 길이가 100cm 이상입니까? (y/n) >> ")
            try:
                if (self.size.lower() == 'y' or self.size.lower() == 'n'):
                    break
                else:
                    print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
            except:
                print("잘못입력하셨습니다. y 혹은 n 으로 입력해주세요.")
        while 1:
            try:
                self.weight_kg = int(input("입력하신 아이템의 개당 무게를 숫자(정수)로 입력하세요. (단위: kg) >> "))
                break
            except:
                print("숫자로 입력해주세요.")
        while 1:
            try:
                self.workload_user = int(input("입력하신 작업 아이템에 대한 작업 갯수를 숫자로 입력하세요. (단위: pcs) >> "))
                break
            except:
                print("숫자로 입력해주세요.")
        self.remark = input("그 외 작업 혹은 아이템에 대한 비고사항을 자유롭게 기재하세요 >> ")
        print("입력해주셔서 감사합니다. 견적서를 출력합니다.")
        
        #작업 아이템이 깨지기 쉽거나 작업하기 힘들면 3 작업하기 보통이면 2 작업하기 간단하면 1
        if (self.fragile == 'y' or self.dangerous == 'y'):
            self.difficulty = 3
        elif self.easy == 'y':
            self.difficulty = 1
        else:
            self.difficulty = 2
         #작업 아이템이 정형화된 박스 모양이면 1 부피가 크거나 길어서 세변의 합이 160cm를 넘으면 2
        if self.size == 'y':
            self.shape = 2
        else:
            self.shape = 1
        #작업 아이템이 20kg이상이면 3, 작업 아이템이 20kg 미만 5kg 이상이면 2, 5kg 미만이면 1
        if self.weight_kg >= 20:
            self.weight = 3
        elif self.weight_kg >= 5:
            self.weight = 2
        else:
            self.weight = 1
        
        # work_definition 의 작업 경우의 수 중에 어떤 것인지 튜플 형식으로 묶고 work_definition 상 어떤 인덱스인지
        self.work_index_user = self.work_definition.index((self.difficulty, self.shape, self.weight))
        self.meandata = self.mean_manhour()
        self.result_s = round(self.meandata[0][self.work_index_user].item()* self.workload_user,1)
        self.result_f = round(self.meandata[1][self.work_index_user].item()* self.workload_user,1)
        self.result_pi = round(self.meandata[2][self.work_index_user].item()* self.workload_user,1)
        self.result_pa = round(self.meandata[3][self.work_index_user].item()* self.workload_user,1)

    def print_quotation(self):
        print(
f"""
----------------------------------------------------------------------------------------
                                        견 적 서


- 견적날짜: {datetime.date.today()}

- 작업 아이템: {self.worktitle}
- 주업무: 피킹/패킹
- 아이템 모양: {"박스모양의 정형화된 아이템" if self.shape == 1 else "크거나 긴 비정형화된 아이템"}
- 아이템 무게/개당: {self.weight}kg
- 아이템 종류: {("위험화물인 배터리, 향수, 알코올, 화학약품 류" if self.dangerous == 'y' 
                else "깨지기 쉬운 유리/그릇 류" if self.fragile == 'y'
                else "비교적 가벼운 플라스틱, 금속, 종이 류" if self.easy == 'y'
                else "작업 강도 중간")}
- 아이템 개수: {self.workload_user}개
- 작업 혹은 아이템에 대한 비고사항: 
    - {self.remark}

- 필요 예상 manhour:
    - Supervisor: {self.result_s} hours
    - Forklift Driver: {self.result_f} hours
    - Picker: {self.result_pi} hours
    - Packer: {self.result_pa} hours
    
----------------------------------------------------------------------------------------

""")

# 견적서 loop
def main():
    end_botton = 1
    while end_botton:
        a = Quotation() # Quotation 객체 생성 후 Quotation 정보 입력
        time.sleep(1)
        a.print_quotation() # Quotation 클래스의 프린트 함수
        time.sleep(1)
        while 1:
            another = input("또 다른 견적서를 원하시면 y 를, 프로그램 종료를 원하시면 n 을 입력하세요 >> ")
            try:
                if another.lower() == 'y':
                    break
                elif another.lower() == 'n':
                    print("프로그램을 종료합니다. 이용해주셔서 감사합니다.")
                    end_botton = 0
                    break
                else:
                    print("잘못입력하셨습니다. y 또는 n 으로 입력해주세요.")
            except:
                print("잘못입력하셨습니다. y 또는 n 으로 입력해주세요.")
        

main()

견적 받고 싶은 아이템에 대하여 8가지 질문을 드릴려고 합니다.                                
최대한 정확한 정보를 입력해주시면 감사하겠습니다.
                                
Pick/Packing 작업 견적을 원하시는 아이템 이름을 입력하세요 >> 냉장고
입력하신 아이템이 깨지기 쉬운 유리/그릇 류 입니까? (y/n) >> n
입력하신 아이템이 위험화물인 배터리, 향수, 알코올, 화학약품 류 입니까? (y/n) >> n
입력하신 아이템이 가벼운 플라스틱, 금속, 종이 류 입니까? (y/n) >> n
입력하신 아이템의 가로+세로+높이 합이 160cm을 넘거나 또는 한변의 길이가 100cm 이상입니까? (y/n) >> y
입력하신 아이템의 개당 무게를 숫자(정수)로 입력하세요. (단위: kg) >> 100
입력하신 작업 아이템에 대한 작업 갯수를 숫자로 입력하세요. (단위: pcs) >> 10
그 외 작업 혹은 아이템에 대한 비고사항을 자유롭게 기재하세요 >> 
입력해주셔서 감사합니다. 견적서를 출력합니다.

----------------------------------------------------------------------------------------
                                        견 적 서


- 견적날짜: 2023-11-10

- 작업 아이템: 냉장고
- 주업무: 피킹/패킹
- 아이템 모양: 크거나 긴 비정형화된 아이템
- 아이템 무게/개당: 3kg
- 아이템 종류: 작업 강도 중간
- 아이템 개수: 10개
- 작업 혹은 아이템에 대한 비고사항: 
    - 

- 필요 예상 manhour:
    - Supervisor: 4.0 hours
    - Forklift Driver: 4.4 hours
    - Picker: 14.3 hours
    - Packer: 14.2 hours
    
------------------------------